In [1]:
import os
import pandas as pd

root_pth = '/home/ddon0001/PhD/experiments/scaled/no_merges_all'
ds_summary_path = os.path.join(root_pth, 'summary.csv')
ds_info = pd.read_csv(ds_summary_path)[['ds_name', 'seg_path', 'tra_gt_path']]
all_ds_path = os.path.join(root_pth, 'solution_edges_datasets_with_FP.csv')
all_ds = pd.read_csv(all_ds_path)

original_metrics_path = '/home/ddon0001/PhD/experiments/as_ctc/no_merges/no_merge_metrics.csv'
original_metrics = pd.read_csv(original_metrics_path, sep=';')

out_root = '/home/ddon0001/PhD/experiments/error_sampling_iter'

In [9]:
from tracktour import load_tiff_frames
from utils import OUT_FILE, join_pth
import networkx as nx
import json

ds_name = 'Fluo-N2DH-GOWT1_01'
feature_of_interest = 'distance'
ascending = False

# select edges and sort by relevant feature
ds_edges = all_ds[all_ds['ds_name'] == ds_name][[
    'ds_name',
    'u',
    'v',
    'distance',
    'chosen_neighbour_rank',
    'prop_diff',
    'sensitivity_diff',
    'solution_correct',
    'solution_incorrect'
]]
ds_edges = ds_edges.sort_values(by=feature_of_interest, ascending=ascending)

# load required dataset info
ds_root_pth = os.path.join(root_pth, ds_name)
solution_graph = nx.read_graphml(join_pth(ds_root_pth, OUT_FILE.MATCHED_SOL), node_type=int)
gt_graph = nx.read_graphml(join_pth(ds_root_pth, OUT_FILE.MATCHED_GT))
gt_path = ds_info[ds_info['ds_name'] == ds_name]['tra_gt_path'].values[0]

with open(join_pth(ds_root_pth, OUT_FILE.MATCHING), 'r') as f:
    node_match = json.load(f)
gt_to_sol = {item[0]: item[1] for item in node_match}
sol_to_gt = {item[1]: item[0] for item in node_match}
original_seg = load_tiff_frames(ds_info[ds_info['ds_name'] == ds_name]['seg_path'].values[0])

# prepare information for copying nodes across
node_copy_info = {}
if len(original_seg.shape) == 4:
    node_copy_info['z'] = 'x'
    node_copy_info['y'] = 'y'
    node_copy_info['x'] = 'z'
    gt_loc = ['x', 'y', 'z']
else:
    node_copy_info['y'] = 'x'
    node_copy_info['x'] = 'y'
    gt_loc = ['x', 'y']
node_copy_info['t'] = 't'
# node_copy_info['label'] = 'segmentation_id'
loc = gt_loc[::-1]
new_label = original_seg.max() + 1
new_node_id = solution_graph.number_of_nodes()
n_digits = max(len(str(len(original_seg))), 3)

ds_metrics = original_metrics[original_metrics['dataset'] == ds_name]
original_lnk = ds_metrics['LNK'].values[0]
original_bio = ds_metrics['BIO(0)'].values[0]
original_ct = ds_metrics['CT'].values[0]
original_tf = ds_metrics['TF'].values[0]
original_cca = ds_metrics['CCA'].values[0]
original_bc = ds_metrics['BC(0)'].values[0]

ds_edges['LNK'] = -1.0
ds_edges['BIO(0)'] = -1.0
ds_edges['CT'] = -1.0
ds_edges['TF'] = -1.0
ds_edges['CCA'] = -1.0
ds_edges['BC(0)'] = -1.0

ds, seq = ds_name.split('_')
out_ds = os.path.join(out_root, ds)
out_res = os.path.join(out_ds, f'{seq}_RES')

Loading TIFFs: 100%|██████████| 83/83 [00:00<00:00, 509.79it/s]


In [10]:
from ctc_metrics.scripts.evaluate import evaluate_sequence
from tifffile import imread
from tracktour._io_util import get_ctc_output
from tracktour.cli import _save_results

for row in ds_edges.itertuples():
    is_correct = row.solution_correct
    u = row.u
    v = row.v
    if not is_correct:
        # get correct destinations for u from gt
        gt_u = sol_to_gt[u]
        dests = list(gt_graph.successors(gt_u))
        for dest in dests:
            # destination exists, we just need to swap the edges
            if dest in gt_to_sol:
                sol_dest = gt_to_sol[dest]
            # destination was a missing vertex
            else:
                # determine node information to copy over
                node_info = gt_graph.nodes[dest]
                copy_info = {}
                for k in node_copy_info:
                    gt_k = node_copy_info[k]
                    copy_info[k] = node_info[gt_k]
                copy_info['label'] = new_label
                to_add = [(new_node_id, copy_info)]
                sol_dest = new_node_id

                # add node to solution graph
                solution_graph.add_nodes_from(to_add)

                # mask segmentation
                t = copy_info['t']
                frame_name = f'man_track{str(t).zfill(n_digits)}.tif'
                frame_path = os.path.join(gt_path, frame_name)
                frame = imread(frame_path)
                mask = frame == node_info['segmentation_id']
                original_seg[t][mask] = new_label

                # increment information for next node to add    
                new_label += 1
                new_node_id += 1

            # add correct edge
            if not solution_graph.has_edge(u, sol_dest):
                solution_graph.add_edge(u, sol_dest)

        if solution_graph.has_edge(u, v):
            # remove wrong edge
            solution_graph.remove_edge(u, v)

        # save CTC output, get new new_label based on actual segmentation
        original_seg, track_df, new_label = get_ctc_output(original_seg, solution_graph, 't', 'label', loc)
        _save_results(original_seg, track_df, out_res)
        new_label += 1

        # re-evaluate LNK and BIO
        res_dict = evaluate_sequence(out_res, gt_path[:-4], ['LNK', 'BIO', 'CT', 'TF', 'CCA', 'BC'])

        # overwrite original metrics
        original_lnk = res_dict['LNK']
        original_bio = res_dict['BIO(0)']
        original_ct = res_dict['CT']
        original_tf = res_dict['TF']
        original_cca = res_dict['CCA']
        original_bc = res_dict['BC(0)']

    # save into graph
    ds_edges.at[row.Index, 'LNK'] = original_lnk
    ds_edges.at[row.Index, 'BIO(0)'] = original_bio
    ds_edges.at[row.Index, 'CT'] = original_ct
    ds_edges.at[row.Index, 'TF'] = original_tf
    ds_edges.at[row.Index, 'CCA'] = original_cca
    ds_edges.at[row.Index, 'BC(0)'] = original_bc
ds_edges.to_csv(os.path.join(out_ds, f'{seq}_edges_{feature_of_interest}.csv'), index=False)


                
    

Evaluate sequence:  /home/ddon0001/PhD/experiments/error_sampling_iter/DIC-C2DH-HeLa/02_RES  with ground truth:  /home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/DIC-C2DH-HeLa/02_GT/with results:  {'LNK': 0.9324101549620838, 'BIO': None, 'CT': 0.36619718309859156, 'TF': 0.8418576977787503, 'CCA': None, 'BC': None, 'BC(0)': 0.33333333333333337, 'BC(1)': 0.33333333333333337, 'BC(2)': 0.33333333333333337, 'BC(3)': 0.33333333333333337, 'BIO(0)': 0.5137960714035584, 'BIO(1)': 0.5137960714035584, 'BIO(2)': 0.5137960714035584, 'BIO(3)': 0.5137960714035584, 'OP_CLB(0)': 0.7231031131828212, 'OP_CLB(1)': 0.7231031131828212, 'OP_CLB(2)': 0.7231031131828212, 'OP_CLB(3)': 0.7231031131828212}  done!
Evaluate sequence:  /home/ddon0001/PhD/experiments/error_sampling_iter/DIC-C2DH-HeLa/02_RES  with ground truth:  /home/ddon0001/PhD/data/cell_tracking_challenge/SUBMISSION/DIC-C2DH-HeLa/02_GT/with results:  {'LNK': 0.9340586877678866, 'BIO': None, 'CT': 0.39436619718309857, 'TF': 0.864934620855

/home/ddon0001/miniconda3/envs/isbi25/lib/python3.10/site-packages/pandas/core/internals/blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [4]:
ds_edges.to_csv(os.path.join(out_ds, f'{seq}_edges_distance.csv'), index=False)

In [ ]:
ds_edges[ds_edges.u == 1337]